In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_context("talk")
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
from collections import Counter
import xgboost as xgb
import pickle

In [2]:
DATA_FOLDER = '../readonly/final_project_data/'

# transactions = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
# items = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
# item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
# shops = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
test = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))


In [3]:
training=pd.read_feather("final_data101217.ft")

In [4]:
training=training[training.columns[~training.columns.str.contains("item_id_")]]

In [5]:
X_sparse=training.drop(columns="item_cnt_day").to_sparse()

In [48]:
#items=pd.get_dummies(X_sparse.item_id,prefix="item_id",sparse=True)

In [49]:
#X=pd.concat([X_sparse,items],axis=1).drop(columns=["shop_id","item_id"]).fillna(0)

In [7]:
X_sparse

,date_block_num,item_id,shop_id,item_price,month,ii,буденовский,век,воронеж,горизонт,...,shop_id_49,shop_id_50,shop_id_51,shop_id_52,shop_id_53,shop_id_55,shop_id_56,shop_id_57,shop_id_58,shop_id_59
0,8,30,21,399.000,9,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
1,8,30,24,399.000,9,False,True,False,False,False,...,0,0,0,0,0,0,0,0,0,0
2,8,30,25,399.000,9,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
3,8,30,28,399.000,9,True,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
4,8,30,31,399.000,9,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
5,8,30,41,399.000,9,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
6,8,30,50,399.000,9,False,False,False,False,False,...,0,1,0,0,0,0,0,0,0,0
7,8,30,53,399.000,9,False,False,False,False,False,...,0,0,0,0,1,0,0,0,0,0
8,8,30,57,399.000,9,False,False,False,False,False,...,0,0,0,0,0,0,0,1,0,0
9,8,31,3,699.000,9,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0


In [55]:
X=X.to_coo().tocsc()

In [109]:
y=training["item_cnt_day"].clip(lower=0,upper=20)

In [110]:
clas=xgb.XGBRegressor(n_jobs=8,n_estimators=200)

In [111]:
clas.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=8, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [112]:
res=clas.predict(X).round()

In [120]:
with open("model.dat","wb") as file:
    pickle.dump(clas,file)

In [131]:
import gc
gc.collect()
gc.collect()

0

In [145]:
metadata=training.groupby(["item_id","shop_id"]).head(1).drop(columns=["date_block_num","month"])

In [144]:
pd.read_feather("final_data101217.ft").query("item_id==5320")

,date_block_num,item_id,shop_id,item_price,month,ii,буденовский,век,воронеж,горизонт,...,item_id_22144,item_id_22145,item_id_22154,item_id_22155,item_id_22158,item_id_22162,item_id_22163,item_id_22164,item_id_22166,item_id_22167


In [146]:
(test
 .assign(date_block_num=34,month=11)
 .drop(columns="ID")
 .merge(metadata,how="left",on=["shop_id","item_id"])
 .drop(columns="shop_id")
)

,item_id,date_block_num,month,item_price,ii,буденовский,век,воронеж,горизонт,жуковский,...,shop_id_49,shop_id_50,shop_id_51,shop_id_52,shop_id_53,shop_id_55,shop_id_56,shop_id_57,shop_id_58,shop_id_59
0,5037,34,11,2599.000,False,False,False,False,False,False,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,5320,34,11,nan,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,5233,34,11,999.000,False,False,False,False,False,False,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,5232,34,11,nan,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,5268,34,11,nan,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,5039,34,11,2599.000,False,False,False,False,False,False,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
6,5041,34,11,3832.333,False,False,False,False,False,False,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
7,5046,34,11,299.000,False,False,False,False,False,False,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
8,5319,34,11,299.000,False,False,False,False,False,False,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
9,5003,34,11,nan,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
